In [1]:
import numpy as np
from scipy.io import loadmat
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Settings

## General settings

In [2]:
settings = {}
settings['cdrtype'] = 0
settings['diffh'] = 2
settings['method'] = 'binaryparametric'

def sett2str(sett):
    sett_str = ''
    for key, value in sett.items():
        sett_str += '%s%s_' % (key, value)
    return sett_str[:-1]

## Path settings

In [3]:
save_path = 'results'
load_path = os.path.join('results', 'timeseries')


# Load timeseries

In [4]:
ts = None

for cdrtype in [1, 2, 3, 4]:
    settings['cdrtype'] = cdrtype
    
    file_path = os.path.join(load_path, 'ts_%s.mat' % sett2str(settings))
    
    if settings['method'] == 'multiparametric':
        ts_i = loadmat(file_path)['ts']
    else:
        ts_i = np.expand_dims(loadmat(file_path)['ts'], axis=2)
    
    if ts is None:
        ts = ts_i
    else:
        ts = np.concatenate((ts, ts_i), axis=2)

y = np.ones((ts.shape[0],))
y[int(y.shape[0]/2):] = 0

print(ts.shape, y.shape)

(20000, 231, 4) (20000,)


# Train-Test split

In [5]:
ts_tr, ts_te, y_tr, y_te = train_test_split(ts, y, test_size=0.2, random_state=1)

# Average

In [66]:
y_pr_s = np.zeros((y_te.shape[0], ts_te.shape[2]))

for dim in tqdm(range(y_pr_s.shape[1])):
    lr = LogisticRegression()

    mean_tr = np.mean(ts_tr[:, :, dim], axis=1).reshape((-1, 1))

    lr.fit(mean_tr, y_tr)

    mean_te = np.mean(ts_te[:, :, dim], axis=1).reshape((-1, 1))

    y_pr_i = lr.predict(mean_te)
        
    y_pr_s[:, dim] = y_pr_i


100%|██████████| 4/4 [00:00<00:00, 30.29it/s]


In [67]:
y_pr = np.mean(y_pr_s, axis=1) > 0.5

acc = np.sum(y_pr == y_te)/y_pr.shape[0]

print(acc)

0.852
